<a href="https://colab.research.google.com/github/ominivac/analise_integraSUS/blob/master/C%C3%B3pia_de_Analise_de_dados_do_IntegraSUS_PLOTLY_Roberto_Sousa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notebook Pandas para Análise Descritiva - concurso INTEGRASUS 2020**
Fonte dos dados analisados :  http://download-integrasus.saude.ce.gov.br/download

Paticipante : Roberto Sousa - robertokbs@gmail.com

In [1]:
import folium
import warnings
import psycopg2
import numpy as np
import pandas as pd
import altair as alt
from datetime import *
import plotly.express as px
import plotly.graph_objects as go



/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/IntegraSUS/casos_coronavirus_2021_01_10.csv", low_memory=False, sep=';', date_parser= '%Y-%m-%d')

In [4]:
#df.info()

In [5]:
#quantidade de registros
print('Nº de registros : ' , df.shape )

Nº de registros :  (1593963, 62)


In [6]:
# Preview dos dados , indentificação de colunas, fortmato dos dados etc
df.head(3)

,identificadorCaso,idRedcap,idEsus,idSivep,classificacaoEstadoRedcap,classificacaoEstadoEsus,classificacaoFinalEsus,evolucaoCasoEsus,cboEsus,profissionalSaudeEsus,paisCaso,estadoCaso,codigoMunicipioCaso,municipioCaso,bairroCaso,bairroCasoGeocoder,sexoCaso,idadeCaso,faixaEtaria,dataNascimento,dataNotificacao,dataInicioSintomas,dataSolicitacaoExame,dataColetaExame,dataResultadoExame,resultadoFinalExame,laboratorioExame,classificacaoEstadoSivep,dataInternacaoSivep,dataEntradaUTISivep,dataSaidaUTISivep,evolucaoCasoSivep,dataEvolucaoCasoSivep,comorbidadePuerperaSivep,comorbidadeCardiovascularSivep,comorbidadeHematologiaSivep,comorbidadeSindromeDownSivep,comorbidadeHepaticaSivep,comorbidadeAsmaSivep,comorbidadeDiabetesSivep,comorbidadeNeurologiaSivep,comorbidadePneumopatiaSivep,comorbidadeImunodeficienciaSivep,comorbidadeRenalSivep,comorbidadeObesidadeSivep,comorbidadeHiv,comorbidadeNeoplasias,tipoTesteExame,requisicaoGal,racaCor,dataNotificacaoObito,cnesNotificacaoEsus,municipioNotificacaoEsus,tipoObitoMaterno,classificacaoFinalCasoSivep,gestante,tipoTesteEsus,obitoConfirmado,dataObito,classificacaoObito,localObito,tipoLocalObito
0,1c3dd697de88933a4be558af6378f22d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CE,230440.0,FORTALEZA,NaN,NaN,FEMININO,31.0,30 a 34 anos,1988-07-02,NaN,NaT,NaN,2020-05-07 00:00:00,2020-05-07 00:00:00,Positivo,HIPOLITO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Teste Rápido,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ff7313a0641ae3ee022b7d85c6554842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CE,230440.0,FORTALEZA,NaN,NaN,MASCULINO,51.0,50 a 54 anos,1968-06-20,NaN,NaT,NaN,2020-05-07 00:00:00,2020-05-07 00:00:00,Positivo,HIPOLITO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Teste Rápido,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1e83e6c50c2e687901bc0a4e5ac94166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CE,230440.0,FORTALEZA,NaN,NaN,MASCULINO,70.0,70 a 74 anos,1950-02-24,NaN,NaT,NaN,2020-05-07 00:00:00,2020-05-07 00:00:00,Positivo,HIPOLITO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Teste Rápido,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.dtypes

identificadorCaso             object
idRedcap                     float64
idEsus                        object
idSivep                      float64
classificacaoEstadoRedcap    float64
                              ...   
obitoConfirmado               object
dataObito                     object
classificacaoObito            object
localObito                    object
tipoLocalObito                object
Length: 62, dtype: object

In [8]:
# Tranformar as col. de data no tipo certo
df[["dataNotificacao","dataSolicitacaoExame","dataResultadoExame","dataColetaExame","dataInicioSintomas", "dataObito", "dataNotificacaoObito"]] =  df[["dataNotificacao","dataSolicitacaoExame","dataResultadoExame","dataColetaExame","dataInicioSintomas", "dataObito", "dataNotificacaoObito"]].apply(pd.to_datetime,  errors='coerce')

In [9]:
df.dtypes

identificadorCaso                    object
idRedcap                            float64
idEsus                               object
idSivep                             float64
classificacaoEstadoRedcap           float64
                                  ...      
obitoConfirmado                      object
dataObito                    datetime64[ns]
classificacaoObito                   object
localObito                           object
tipoLocalObito                       object
Length: 62, dtype: object

**1.1 Distribuição de atributos**

In [10]:
# Algumas colunas em brancos, contar para sexoPaciente
count = df["sexoCaso"].isnull().sum()
print(count)

4129


In [11]:
 # Ver distribuição de atributos
df['racaCor'].unique().tolist()

[nan,
 'Amarela',
 'Parda',
 'Branca',
 'Sem Informacao',
 'Ignorado',
 'Preta',
 'Indígena']

In [12]:
# Alterar onde for nan para Não Informado
df['racaCor'] = df['racaCor'].fillna('Não Informado')

In [13]:
df['tipoTesteExame'].unique().tolist()

['Teste Rápido',
 'RT-PCR',
 'Quimioluminescência',
 'ELISA',
 'Eletroquimioluminescência',
 'Imunocromatografia',
 nan,
 'Não Informado',
 'ECLIA',
 'Confirmado Clínico-Epidemiológico',
 'Confirmado por Critério Clínico',
 'CLIA',
 'Confirmado Clínico-Imagem']

In [14]:
df['evolucaoCasoSivep'].unique().tolist()

[nan, 'Cura', '3', 'Ignorado', 'Óbito']

In [15]:
df['comorbidadePneumopatiaSivep'].unique().tolist()

[nan, 'Não', 'Ignorado', 'Sim']

In [16]:
df['tipoTesteExame'].unique().tolist()

['Teste Rápido',
 'RT-PCR',
 'Quimioluminescência',
 'ELISA',
 'Eletroquimioluminescência',
 'Imunocromatografia',
 nan,
 'Não Informado',
 'ECLIA',
 'Confirmado Clínico-Epidemiológico',
 'Confirmado por Critério Clínico',
 'CLIA',
 'Confirmado Clínico-Imagem']

In [17]:
df['resultadoFinalExame'].unique().tolist()

['Positivo', 'Negativo', 'Inconclusivo', 'Em Análise', nan, 'Provável']

In [18]:
df['evolucaoCasoEsus'].unique().tolist()

[nan,
 'Em tratamento domiciliar',
 'Cura',
 'Óbito',
 'Ignorado',
 'Internado',
 'Internado em UTI']

**1.2 - CRIAÇÃO DE NOVAS COLUNAS PARA FACILITAÇÃO DE CÁLCULOS POSTERIORES**

In [19]:
# Criação da coluna faixa de idade para calculo em percentual
# Não á mais necessidade, o dataset já vem com uma coluna de faixa etária

#bins= [0,4,9,14,19,24,29,34,39,44,49,54,59,64,69,74,79,80]
#labels = ['0-4', '5-9', '10-14','15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69 ','70-74', '75-79', '80+']

#df['faixaIdade'] = pd.cut(df['idadeCaso'], bins=bins, labels=labels)
#df.head(3)

**O que define o óbito?** 

In [20]:
# numero oficial : 9.933

 
#df[df['obitoConfirmado'] == True]                                                                                               11658   rows  01 -
#df[df['dataObito'].notnull()]                                                                                                   14738   rows  02 - 
#df[df['evolucaoCasoSivep'] == 'Óbito']                                                                                          11332 	 rows  03 - o mais perto do oficial
#df[ (df['obitoConfirmado'] == True) & (df['dataObito'].notnull())  ]                                                            11658 #df[ (df['dataObito'].notnull() ) & (df['resultadoFinalExame'] == 'Positivo' )]                          11659 rows
#df[ (df['obitoConfirmado'] == True) & (df['dataObito'].notnull() ) & (df['resultadoFinalExame'] == 'Positivo')   ]              11658 combinar 04 e 05 da o mesmo result. 



**Recuperados : casos ja´confirmados e que não evoluiram pra óbito**

In [21]:
# df[(df['resultadoFinalExame'] == 'Positivo') & (df['dataObito'].isnull()  )  ]      # 402.102
# df[(df['evolucaoCasoEsus'] == 'Cura') ]                                            # 478.215 / 265.528 
# df[(df['resultadoFinalExame'] == 'Positivo') & (df['evolucaoCasoEsus'] == 'Cura'  )  ] # 179.657 rows / 265.528       #USAR ESSE

**Casos confirmados - PODEM OU NÃO TER EVOLUIDO PRA ÓBITOS**

In [22]:
#df[(df['resultadoFinalExame'] == 'Positivo')] # 413.761 rows / 327.865 IntegraSUS

In [23]:
#df.sort_values(by="dataObito")

**Nova coluna quantidade de óbito para cada dia, visto que em um mesmo dia há mais de um óbito**

In [24]:
# Como cada dia representa um óbito, criar nova coluna para cada registro somente HUM por registro 
df['qtObito'] = np.where(df['obitoConfirmado'] == True, 1, 0)
#df[df['qtObito'] == 1].head(3)

**Nova coluna quantidade de casos confirmados para cada dia, visto que em um mesmo dia há mais de um caso confirmado**

In [25]:
# Nova coluna para somar de casos confirmados SEM óbito
df['qtCasoConfirmado'] = np.where(  ( df['resultadoFinalExame'] == 'Positivo')  , 1, 0) 
#df[df['qtCasoConfirmado'] == 1].head(3)

**Nova coluna quantidade de casos confirmados SEM ÓBITO**

In [26]:
# Nova coluna para somar de casos confirmados SEM óbito
df['qtCasoConfirmadoSemObito'] = np.where(  ( df['resultadoFinalExame'] == 'Positivo')  & ( df['evolucaoCasoEsus'] == 'Cura' ) , 1, 0) 
#df[df['qtCasoConfirmadoSemObito'] == 1].head(3)

**1.3 - CRIAÇÃO DE NOVO DATASET PARA CASOS CONFIRMADOS QUE PODEM OU NÃO TER EVOLUÍDOS PRA ÓBITOS**

In [27]:
df_confirmados = df[ df['resultadoFinalExame'] == 'Positivo'  ]
df_confirmados.sort_values(by="dataObito").head(3)

,identificadorCaso,idRedcap,idEsus,idSivep,classificacaoEstadoRedcap,classificacaoEstadoEsus,classificacaoFinalEsus,evolucaoCasoEsus,cboEsus,profissionalSaudeEsus,paisCaso,estadoCaso,codigoMunicipioCaso,municipioCaso,bairroCaso,bairroCasoGeocoder,sexoCaso,idadeCaso,faixaEtaria,dataNascimento,dataNotificacao,dataInicioSintomas,dataSolicitacaoExame,dataColetaExame,dataResultadoExame,resultadoFinalExame,laboratorioExame,classificacaoEstadoSivep,dataInternacaoSivep,dataEntradaUTISivep,dataSaidaUTISivep,evolucaoCasoSivep,dataEvolucaoCasoSivep,comorbidadePuerperaSivep,comorbidadeCardiovascularSivep,comorbidadeHematologiaSivep,comorbidadeSindromeDownSivep,comorbidadeHepaticaSivep,comorbidadeAsmaSivep,comorbidadeDiabetesSivep,comorbidadeNeurologiaSivep,comorbidadePneumopatiaSivep,comorbidadeImunodeficienciaSivep,comorbidadeRenalSivep,comorbidadeObesidadeSivep,comorbidadeHiv,comorbidadeNeoplasias,tipoTesteExame,requisicaoGal,racaCor,dataNotificacaoObito,cnesNotificacaoEsus,municipioNotificacaoEsus,tipoObitoMaterno,classificacaoFinalCasoSivep,gestante,tipoTesteEsus,obitoConfirmado,dataObito,classificacaoObito,localObito,tipoLocalObito,qtObito,qtCasoConfirmado,qtCasoConfirmadoSemObito
483504,e00481f300496b7c996f781c5b9edc0f,NaN,202017481,3.158525e+11,NaN,Positivo,Confirmação Laboratorial,Óbito,NaN,False,BRASIL,CE,230440.0,FORTALEZA,COCO,NaN,FEMININO,84.0,80 ou mais,1936-02-26,2020-03-24,2020-03-21,NaT,2020-03-24,2020-03-24,Positivo,GAL,Positivo,NaN,NaN,NaN,Óbito,2020-03-24 00:00:00.000,Não,Sim,Não,Não,Não,Não,Sim,Não,Não,Não,Não,Não,NaN,NaN,RT-PCR,2.022130e+11,Não Informado,2020-05-20 17:24:35.783,NaN,FORTALEZA,NaN,SRAG por COVID-19,NaN,RT-PCR,True,2020-03-24,Confirmado,HOSPITAL SAO MATEUS,NaN,1,1,0
483497,82563a758fad9d2126eabf2d356277d6,NaN,202017950,3.158514e+11,NaN,Positivo,Confirmação Laboratorial,NaN,NaN,False,BRASIL,CE,230440.0,FORTALEZA,JOVITA FEITOSA,NaN,FEMININO,97.0,80 ou mais,1923-02-03,2020-03-24,2020-03-21,NaT,2020-03-24,2020-04-28,Positivo,GAL,Positivo,2020-03-23 00:00:00.000,2020-03-23 00:00:00.000,NaN,Óbito,2020-03-24 00:00:00.000,NaN,Sim,NaN,NaN,NaN,NaN,Sim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RT-PCR,2.022200e+11,Não Informado,2020-05-20 17:30:03.817,NaN,FORTALEZA,NaN,SRAG por COVID-19,NaN,RT-PCR,True,2020-03-24,Confirmado,HOSPITAL GENESIS,NaN,1,1,0
387717,82563a758fad9d2126eabf2d356277d6,NaN,202017974,3.158514e+11,NaN,NaN,NaN,NaN,NaN,False,BRASIL,CE,230440.0,FORTALEZA,JOVITA FEITOSA,NaN,FEMININO,97.0,80 ou mais,1923-02-03,2020-03-24,2020-03-21,NaT,2020-03-24,2020-04-28,Positivo,GAL,Positivo,2020-03-23 00:00:00.000,2020-03-23 00:00:00.000,NaN,Óbito,2020-03-24 00:00:00.000,NaN,Sim,NaN,NaN,NaN,NaN,Sim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RT-PCR,2.022200e+11,Não Informado,2020-05-20 17:30:03.817,NaN,FORTALEZA,NaN,SRAG por COVID-19,NaN,NaN,True,2020-03-24,Confirmado,HOSPITAL GENESIS,NaN,1,1,0


In [28]:
df_confirmados.shape       # 440.395  / 327.865 INTEGRASUS   15/jan

(440395, 65)

**1.4 - CRIAÇÃO DE NOVO DATASET PARA CASOS CONFIRMADOS COM ÓBITO**

In [29]:
df_obitos = df[ df['obitoConfirmado'] == True]
df_obitos.sort_values(by="dataObito").head(3)

,identificadorCaso,idRedcap,idEsus,idSivep,classificacaoEstadoRedcap,classificacaoEstadoEsus,classificacaoFinalEsus,evolucaoCasoEsus,cboEsus,profissionalSaudeEsus,paisCaso,estadoCaso,codigoMunicipioCaso,municipioCaso,bairroCaso,bairroCasoGeocoder,sexoCaso,idadeCaso,faixaEtaria,dataNascimento,dataNotificacao,dataInicioSintomas,dataSolicitacaoExame,dataColetaExame,dataResultadoExame,resultadoFinalExame,laboratorioExame,classificacaoEstadoSivep,dataInternacaoSivep,dataEntradaUTISivep,dataSaidaUTISivep,evolucaoCasoSivep,dataEvolucaoCasoSivep,comorbidadePuerperaSivep,comorbidadeCardiovascularSivep,comorbidadeHematologiaSivep,comorbidadeSindromeDownSivep,comorbidadeHepaticaSivep,comorbidadeAsmaSivep,comorbidadeDiabetesSivep,comorbidadeNeurologiaSivep,comorbidadePneumopatiaSivep,comorbidadeImunodeficienciaSivep,comorbidadeRenalSivep,comorbidadeObesidadeSivep,comorbidadeHiv,comorbidadeNeoplasias,tipoTesteExame,requisicaoGal,racaCor,dataNotificacaoObito,cnesNotificacaoEsus,municipioNotificacaoEsus,tipoObitoMaterno,classificacaoFinalCasoSivep,gestante,tipoTesteEsus,obitoConfirmado,dataObito,classificacaoObito,localObito,tipoLocalObito,qtObito,qtCasoConfirmado,qtCasoConfirmadoSemObito
29757,ed2d67f326d60173a53ee585606b5910,NaN,202016773,3.158931e+11,NaN,Positivo,Confirmação Laboratorial,Óbito,NaN,False,BRASIL,CE,230440.0,FORTALEZA,FATIMA,NaN,FEMININO,41.0,40 a 44 anos,1979-02-04,2020-03-23,2020-03-18,NaT,2020-03-23,2020-03-23,Positivo,GAL,Positivo,2020-03-23 00:00:00.000,NaN,NaN,Óbito,2020-03-24 00:00:00.000,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,NaN,NaN,RT-PCR,2.001430e+11,Não Informado,2020-05-20 17:24:52.363,NaN,FORTALEZA,NaN,SRAG por COVID-19,NaN,RT-PCR,True,2020-03-24,Confirmado,HOSPITAL REGIONAL UNIMED,PRIVADA,1,1,0
483497,82563a758fad9d2126eabf2d356277d6,NaN,202017950,3.158514e+11,NaN,Positivo,Confirmação Laboratorial,NaN,NaN,False,BRASIL,CE,230440.0,FORTALEZA,JOVITA FEITOSA,NaN,FEMININO,97.0,80 ou mais,1923-02-03,2020-03-24,2020-03-21,NaT,2020-03-24,2020-04-28,Positivo,GAL,Positivo,2020-03-23 00:00:00.000,2020-03-23 00:00:00.000,NaN,Óbito,2020-03-24 00:00:00.000,NaN,Sim,NaN,NaN,NaN,NaN,Sim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RT-PCR,2.022200e+11,Não Informado,2020-05-20 17:30:03.817,NaN,FORTALEZA,NaN,SRAG por COVID-19,NaN,RT-PCR,True,2020-03-24,Confirmado,HOSPITAL GENESIS,NaN,1,1,0
387717,82563a758fad9d2126eabf2d356277d6,NaN,202017974,3.158514e+11,NaN,NaN,NaN,NaN,NaN,False,BRASIL,CE,230440.0,FORTALEZA,JOVITA FEITOSA,NaN,FEMININO,97.0,80 ou mais,1923-02-03,2020-03-24,2020-03-21,NaT,2020-03-24,2020-04-28,Positivo,GAL,Positivo,2020-03-23 00:00:00.000,2020-03-23 00:00:00.000,NaN,Óbito,2020-03-24 00:00:00.000,NaN,Sim,NaN,NaN,NaN,NaN,Sim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RT-PCR,2.022200e+11,Não Informado,2020-05-20 17:30:03.817,NaN,FORTALEZA,NaN,SRAG por COVID-19,NaN,NaN,True,2020-03-24,Confirmado,HOSPITAL GENESIS,NaN,1,1,0


In [30]:
df_obitos.shape   # 11.852 óbitos

(11852, 65)

In [31]:
df_grafico_obito = df_obitos.groupby('dataObito', sort=False)["qtObito"].sum().reset_index(name ='total_obito_dia')
df_grafico_obito.sort_values('dataObito')

,dataObito,total_obito_dia
39,2020-03-24,4
287,2020-03-25,2
37,2020-03-26,4
277,2020-03-27,3
289,2020-03-28,2
...,...,...
266,2021-01-04,7
264,2021-01-05,16
267,2021-01-06,6
290,2021-01-07,2


In [32]:
df_gr_obito = (pd.to_datetime(df_obitos['dataObito'])
       .dt.floor('d')
       .value_counts()
       .rename_axis('data_obito')
       .reset_index(name='total_obito_dia'))

df_gr_obito.sort_values('data_obito')

,data_obito,total_obito_dia
282,2020-03-24,4
287,2020-03-25,2
283,2020-03-26,4
286,2020-03-27,3
289,2020-03-28,2
...,...,...
266,2021-01-04,7
173,2021-01-05,16
275,2021-01-06,6
290,2021-01-07,2


In [33]:
df_gr_obito['MA'] = df_gr_obito.rolling(window=7).mean()


In [34]:
df_gr_obito.sort_values('data_obito', inplace= True)

In [36]:
from plotly.subplots import make_subplots
# Plot 
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig = px.bar(df_gr_obito, x='data_obito', y='total_obito_dia',)

# Only thing I figured is - I could do this 
#fig.add_scatter(x=df_gr_conf['data_result_exame'], y= df_gr_conf['MA'])

# Show plot 

fig.add_trace(go.Scatter(x=df_gr_obito['data_obito'], y= df_gr_obito['MA'], name='Média Móvel de óbitos de uma semana',
                        line=dict(color='red',width=2)))

fig.update_layout(title='Quantidade de óbitos confirmados no período',
                   xaxis_title='Mês',
                   yaxis_title='Quantidade de óbitos / dia')
fig.show()

**1.5 Gráfico de óbitos por dia**

In [37]:
fig = go.Figure(data=[
    go.Bar(x= df_grafico_obito['dataObito'], y= df_grafico_obito['total_obito_dia'], marker=dict(color='orangered')),
])

# gráfico
fig.update_layout(
    title='Total de óbitos de Covid-19 por dia no estado do Ceará',
    xaxis_tickfont_size=24,
    xaxis = dict(
        title='Período',
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis = dict(
        title='Número de óbitos',
        titlefont_size=16,
        tickfont_size=14,
    ), xaxis_tickangle=90)

fig.show()

**Metodo para agrupar casos confirmados por dia**

In [38]:
# https://stackoverflow.com/questions/48961892/python-pandas-group-by-day-and-count-for-each-day

df_gr_conf = (pd.to_datetime(df_confirmados['dataResultadoExame'])
       .dt.floor('d')
       .value_counts()
       .rename_axis('data_result_exame')
       .reset_index(name='count'))
#print (df_gr_conf)
df_gr_conf.sort_values('data_result_exame')

,data_result_exame,count
319,2020-01-01,5
308,2020-01-02,7
314,2020-01-03,5
301,2020-01-04,18
304,2020-01-05,15
...,...,...
194,2021-01-06,944
163,2021-01-07,1165
187,2021-01-08,1027
278,2021-01-09,213


In [39]:
df_gr_conf['MA'] = df_gr_conf.rolling(window=7).mean()
df_gr_conf.sort_values('data_result_exame', inplace= True)

In [40]:
df_gr_conf

,data_result_exame,count,MA
319,2020-01-01,5,5.000000
308,2020-01-02,7,12.142857
314,2020-01-03,5,5.857143
301,2020-01-04,18,39.428571
304,2020-01-05,15,22.285714
...,...,...,...
194,2021-01-06,944,976.428571
163,2021-01-07,1165,1183.571429
187,2021-01-08,1027,1046.000000
278,2021-01-09,213,237.285714


In [41]:
from plotly.subplots import make_subplots
# Plot 
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig = px.bar(df_gr_conf, x='data_result_exame', y='count',)

# Only thing I figured is - I could do this 
#fig.add_scatter(x=df_gr_conf['data_result_exame'], y= df_gr_conf['MA'])

# Show plot 

fig.add_trace(go.Scatter(x=df_gr_conf['data_result_exame'], y= df_gr_conf['MA'], name='Média Móvel de uma semana',
                        line=dict(color='red',width=2)))

fig.update_layout(title='Quantidade de casos confirmados no período',
                   xaxis_title='Mês',
                   yaxis_title='Quantidade de casos / dia')
fig.show()

**1.4 Gráfico casos confirmados por dia**

**Metodo para agrupar por dia a quantidade de casos confirmados**

In [43]:
df_gr_obito.head(2)

,data_obito,total_obito_dia,MA
282,2020-03-24,4,4.428571
287,2020-03-25,2,3.428571


In [44]:
df_gr_obito['total_obito_acumulado'] = df_gr_obito['total_obito_dia'].cumsum()
df_gr_obito

,data_obito,total_obito_dia,MA,total_obito_acumulado
282,2020-03-24,4,4.428571,4
287,2020-03-25,2,3.428571,6
283,2020-03-26,4,4.285714,10
286,2020-03-27,3,3.714286,13
289,2020-03-28,2,2.857143,15
...,...,...,...,...
266,2021-01-04,7,7.142857,11826
173,2021-01-05,16,16.714286,11842
275,2021-01-06,6,6.000000,11848
290,2021-01-07,2,2.571429,11850


**GERAR GRÁFICO DE ÓBITOS E CASOS CONFIRMADOS POR DIA**

**Gerar grupo de faixa de idade por óbito**

In [45]:
df_obitos.head(2)

,identificadorCaso,idRedcap,idEsus,idSivep,classificacaoEstadoRedcap,classificacaoEstadoEsus,classificacaoFinalEsus,evolucaoCasoEsus,cboEsus,profissionalSaudeEsus,paisCaso,estadoCaso,codigoMunicipioCaso,municipioCaso,bairroCaso,bairroCasoGeocoder,sexoCaso,idadeCaso,faixaEtaria,dataNascimento,dataNotificacao,dataInicioSintomas,dataSolicitacaoExame,dataColetaExame,dataResultadoExame,resultadoFinalExame,laboratorioExame,classificacaoEstadoSivep,dataInternacaoSivep,dataEntradaUTISivep,dataSaidaUTISivep,evolucaoCasoSivep,dataEvolucaoCasoSivep,comorbidadePuerperaSivep,comorbidadeCardiovascularSivep,comorbidadeHematologiaSivep,comorbidadeSindromeDownSivep,comorbidadeHepaticaSivep,comorbidadeAsmaSivep,comorbidadeDiabetesSivep,comorbidadeNeurologiaSivep,comorbidadePneumopatiaSivep,comorbidadeImunodeficienciaSivep,comorbidadeRenalSivep,comorbidadeObesidadeSivep,comorbidadeHiv,comorbidadeNeoplasias,tipoTesteExame,requisicaoGal,racaCor,dataNotificacaoObito,cnesNotificacaoEsus,municipioNotificacaoEsus,tipoObitoMaterno,classificacaoFinalCasoSivep,gestante,tipoTesteEsus,obitoConfirmado,dataObito,classificacaoObito,localObito,tipoLocalObito,qtObito,qtCasoConfirmado,qtCasoConfirmadoSemObito
4626,d569b935b5fa58b42fbd5da16ea888bb,NaN,NaN,3.160216e+11,NaN,NaN,NaN,NaN,NaN,NaN,BRASIL,CE,230030.0,ACOPIARA,NaN,NaN,FEMININO,75.0,75 a 79 anos,1945-11-03,2020-10-08,2020-10-08,2020-10-08,2020-10-08 03:00:00,2020-10-11 14:28:23,Positivo,LACEN,Negativo,NaN,NaN,NaN,3,2020-10-08 00:00:00.000,Não,Sim,Não,Não,Sim,Não,Sim,Não,Não,Não,Não,Não,NaN,NaN,Teste Rápido,NaN,Branca,2020-10-12 10:27:50.673,NaN,NaN,NaN,NaN,NaN,NaN,True,2020-10-08,SRAG Teste Rápido +,NaN,DOMICILIAR,1,1,0
12172,8824689678bbe6573bc5788aa4c01956,NaN,NaN,3.160683e+11,NaN,NaN,NaN,NaN,NaN,NaN,BRASIL,CE,230640.0,ITAPIPOCA,ESTACAO,NaN,MASCULINO,62.0,60 a 64 anos,1958-12-05,2020-11-30,2020-11-27,2020-12-01,2020-11-30 06:00:00,2020-12-03 19:08:17,Positivo,LACEN,Em Análise,2020-11-30 00:00:00.000,NaN,NaN,Ignorado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Teste Rápido,NaN,Parda,2020-12-02 11:07:50.430,NaN,NaN,NaN,SRAG por COVID-19,NaN,NaN,True,2020-12-01,SRAG Teste Rápido +,CENTRO DE ONCOLOGIA LEONARDO DA VINCI,PRIVADA,1,1,0


In [47]:
counts_obitos_faixa_idade = df_obitos.groupby(['faixaEtaria', 'sexoCaso']).faixaEtaria.count().unstack()
counts_obitos_faixa_idade

sexoCaso,FEMININO,MASCULINO
faixaEtaria,,
00 a 04 anos,23,13
05 a 09 anos,2,5
10 a 14 anos,3,6
15 a 19 anos,14,20
20 a 24 anos,30,22
25 a 29 anos,29,53
30 a 34 anos,53,73
35 a 39 anos,83,142
40 a 44 anos,95,208


In [ ]:
#type(counts_obitos_faixa_idade)

In [60]:
df1 = counts_obitos_faixa_idade.reset_index()
df1


sexoCaso,faixaEtaria,FEMININO,MASCULINO
0,00 a 04 anos,23,13
1,05 a 09 anos,2,5
2,10 a 14 anos,3,6
3,15 a 19 anos,14,20
4,20 a 24 anos,30,22
5,25 a 29 anos,29,53
6,30 a 34 anos,53,73
7,35 a 39 anos,83,142
8,40 a 44 anos,95,208
9,45 a 49 anos,139,245


In [49]:
import plotly.express as px

sexos = ['MASCULINO', 'FEMININO']
faixas  = ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49',
           '50-54', '55-59', '60-64', '65-69 ', '70-74', '75-79', '80+']



fig = go.Figure(data=[
    go.Bar(name='MASCULINO', x= faixas, y= df1['MASCULINO']),
    go.Bar(name='FEMININO', x= faixas, y= df1['FEMININO'])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(title_text='Total de óbitos por faixa de idade')
fig.show()





Distribuição de óbitos por sexo do paciente

In [50]:
obito_by_sexo = df_obitos.groupby([ 'sexoCaso', 'racaCor']).size().unstack('racaCor')
obito_by_sexo.columns =  ['Parda','Branca','Amarela', 'Não Informado','Preta','Ignorado','Indígena', 'Indigena']

In [51]:
obito_by_sexo

,Parda,Branca,Amarela,Não Informado,Preta,Ignorado,Indígena,Indigena
sexoCaso,,,,,,,,
FEMININO,48,391,10,2,2073,2377,54,22
MASCULINO,71,448,8,12,2767,3444,101,24


In [52]:
obito_by_sexo['ind'] = obito_by_sexo['Indígena']+obito_by_sexo['Indigena']
obito_by_sexo

,Parda,Branca,Amarela,Não Informado,Preta,Ignorado,Indígena,Indigena,ind
sexoCaso,,,,,,,,,
FEMININO,48,391,10,2,2073,2377,54,22,76
MASCULINO,71,448,8,12,2767,3444,101,24,125


In [53]:
obito_by_sexo.drop(['Indígena', 'Indigena'], axis=1, inplace= True)
obito_by_sexo

,Parda,Branca,Amarela,Não Informado,Preta,Ignorado,ind
sexoCaso,,,,,,,
FEMININO,48,391,10,2,2073,2377,76
MASCULINO,71,448,8,12,2767,3444,125


In [54]:
obito_by_sexo.rename(columns={"ind": "Indigena"}, inplace= True)
obito_by_sexo

,Parda,Branca,Amarela,Não Informado,Preta,Ignorado,Indigena
sexoCaso,,,,,,,
FEMININO,48,391,10,2,2073,2377,76
MASCULINO,71,448,8,12,2767,3444,125


In [55]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

labels = ['FEMININO', 'MASCULINO']

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=2, cols=3, specs=[ [ {'type':'domain'}, {'type':'domain'}, {'type':'domain'}], 
                                          [{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]
                                          ]
                    )

#parda
fig.add_trace(go.Pie(labels=labels, values=[45,68], name="Parda"), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=[381,441], name="Branca"), 1, 2)
fig.add_trace(go.Pie(labels=labels, values=[9,6], name="Amarela"), 1, 3)

fig.add_trace(go.Pie(labels=labels, values=[2,12], name="Não Informado"), 2, 1)
fig.add_trace(go.Pie(labels=labels, values=[9,6], name="Preta"), 2, 2)
fig.add_trace(go.Pie(labels=labels, values=[381,441], name="Ignorado"),2, 3)

#fig.add_trace(go.Pie(labels=labels, values=[381,441], name="Ignorado"),3, 1)
#branca

#fig.add_trace(go.Pie(labels=labels, values=[68, 381], name="Parda"), 2, 1)
#fig.add_trace(go.Pie(labels=labels, values=[68, 441], name="Branca"),2, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4) #hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Relação de Número de óbitos confirmados segundo raça/cor",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Parda', x=0.12, y=0.5, font_size=20, showarrow=False),
                 dict(text='Branca', x=0.50, y=0.5, font_size=20, showarrow=False),
                 dict(text='Amarela', x=0.90, y=0.5, font_size=20, showarrow=False),
                  dict(text='Nao Informado', x=0.12, y=3.4, font_size=20, showarrow=False)]
                  )
fig.show()



In [57]:
counts_casos_raca_cor = df_obitos.groupby(['racaCor', 'sexoCaso']).racaCor.count().unstack()
counts_casos_raca_cor

sexoCaso,FEMININO,MASCULINO
racaCor,,
Amarela,48,71
Branca,391,448
Ignorado,10,8
Indígena,2,12
Não Informado,2073,2767
Parda,2377,3444
Preta,54,101
Sem Informacao,22,24


In [61]:
df_counts_casos_raca_cor = counts_casos_raca_cor.reset_index()
df_counts_casos_raca_cor

sexoCaso,racaCor,FEMININO,MASCULINO
0,Amarela,48,71
1,Branca,391,448
2,Ignorado,10,8
3,Indígena,2,12
4,Não Informado,2073,2767
5,Parda,2377,3444
6,Preta,54,101
7,Sem Informacao,22,24


In [64]:
import plotly.express as px

sexos = ['MASCULINO', 'FEMININO']
faixas  = ['Amarela	', 'Branca', 'Ignorado', 'Indígena', 'Não Informado', 'Parda', 'Preta', 'Sem Informação']



fig = go.Figure(data=[
    go.Bar(name='MASCULINO', x= df_counts_casos_raca_cor.racaCor	, y= df_counts_casos_raca_cor['MASCULINO']),
    go.Bar(name='FEMININO', x= df_counts_casos_raca_cor.racaCor, y= df_counts_casos_raca_cor['FEMININO'])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(title_text='Total de óbitos por faixa de idade')
fig.show()

In [65]:
df_confirmados_por_municipio = pd.read_csv("/content/drive/My Drive/IntegraSUS/casos_coronavirus_localidade_2021_01_10.csv", low_memory=False, sep=';')

In [66]:
df_confirmados_por_municipio.head(2)

,paisCaso,estadoCaso,municipioCaso,codigoMunicipioCaso,bairroCaso,cepCaso
0,NaN,CE,FORTALEZA,230440.0,BARRA DO CEARA,60332690.0
1,NaN,CE,FORTALEZA,230440.0,NaN,NaN


In [ ]:
# Como cada linha representa um caso, criar nova coluna com quantidade um para posterior agrupamento 
df_confirmados_por_municipio['qt_caso'] = np.where(df['municipioCaso'].notnull, 1, 0)


In [ ]:
df_confirmados_por_municipio.head(3)

In [ ]:
df_grafico_confirmados_municipio = df_confirmados_por_municipio.groupby('municipioCaso', sort=False)["qt_caso"].sum().reset_index(name ='total_caso_municipio')

In [ ]:
df_grafico_confirmados_municipio.sort_values('total_caso_municipio')
df_grafico_confirmados_municipio

In [ ]:
type(df_grafico_confirmados_municipio)

In [ ]:
df_grafico_confirmados_municipio[df_grafico_confirmados_municipio['municipioCaso'] == 'FORTALEZA']

In [ ]:


fig = go.Figure(data=[
    go.Bar(x= df_grafico_confirmados_municipio['municipioCaso'], y=  df_grafico_confirmados_municipio['total_caso_municipio'], marker=dict(color='orangered')),
])

# gráfico
fig.update_layout(
    title='Total de casos confiramdos distribuiídos por raça do paciente no estado do Ceará',
    xaxis_tickfont_size=24,
    xaxis = dict(
        title='Município',
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis = dict(
        title='Número de casos confirmados',
        titlefont_size=16,
        tickfont_size=14,
    ), xaxis_tickangle=90)

fig.show()

In [ ]:
import plotly.express as px


fig = px.bar(df_grafico_confirmados_municipio, x= df_grafico_confirmados_municipio['municipioCaso'], y= df_grafico_confirmados_municipio['total_caso_municipio'])
fig.show()